
# Cassava Leaf Disease Classification

Nesta competição, tenta-se identificar doenças comuns em plantações de mandioca usando ciência de dados e aprendizado de máquina. Métodos de detecção de doenças exigem que os agricultores solicitem a ajuda de especialistas agrícolas financiados pelo governo para inspecionar visualmente e diagnosticar as plantas. Isso sofre por ser muito trabalhoso, com baixo suprimento e caro. Em vez disso, seria preferível se um pipeline automatizado baseado em fotos de qualidade móvel das folhas de mandioca pudesse ser desenvolvido.

Nesse notebook,  será utilizado um conjunto de dados disponível em https://www.kaggle.com/c/cassava-leaf-disease-classification/leaderboard, rotulado por especialistas do National Crops Resources Research Institute (NaCRRI).

Neste kernel, é usado um iniciador fastai.

## Olhando os dados

Para começar, foi configurado o ambiente, instalando e importando os módulos necessários e definindo uma semente aleatória:

In [ ]:
!pip install ../input/pytorch-image-models/timm-0.3.1-py3-none-any.whl

In [ ]:
import numpy as np
import os
import pandas as pd
from fastai.vision.all import *

In [ ]:
set_seed(999)

Verificando o que está disponível:

In [ ]:
dataset_path = Path('../input/cassava-leaf-disease-classification')
os.listdir(dataset_path)

Verifica-se que um arquivo csv de treinamento (train.csv) contém os nomes e rótulos de imagem. O csv de envio de amostra, com os nomes de imagem de teste e as pastas de imagem de teste e treinamento. Também há as imagens no formato tfrecords, que é útil para o carregamento rápido de imagens, especialmente para TensorFlow e TPUs. Não usado nesse notebook.

Verificando o arquivo csv de treinamento e removendo a imagens duplicadas de acordo com esta [discussão](https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198202)

'1562043567.jpg' e '3551135685.jpg' (rótulo incorreto)

'2252529694.jpg' e '911861181.jpg' (duplicado)

In [ ]:
data = pd.read_csv(dataset_path/'train.csv')
train_df = data[~data['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg'])]

In [ ]:
train_df.head()

Executando um processamento rápido dos nomes dos arquivos de imagem para facilitar o acesso:

In [ ]:
train_df['path'] = train_df['image_id'].map(lambda x:dataset_path/'train_images'/x)
train_df = train_df.drop(columns=['image_id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #mix dataframe
train_df.head(5)

Verificando quantas imagens estão disponíveis no conjunto de dados de treinamento:

In [ ]:
len_df = len(train_df)
print(f"Dataset contém {len_df} imagens")


Há um conjunto com com mais de 21k imagens! Com isso é possível desenvolver um modelo preditivo, robusto e generalizável com este conjunto de dados.

Agora, verificando a distribuição das diferentes classes:

In [ ]:
with open(dataset_path/"label_num_to_disease_map.json") as f:
    class_names = json.loads(f.read())
f.close()

train_df["label_name"] = train_df['label'].apply(lambda x: class_names[str(x)])
train_df.label = train_df.label.astype(str)

print("Total exemplos de treino: ", len(train_df))
train_df.head(10)

In [ ]:
count = train_df.label.value_counts().sort_index()
plt.bar(count.keys(), count)

Neste caso, temos 5 rótulos (4 doenças e saudável):
0. Cassava Bacterial Blight (CBB)
1. Cassava Brown Streak Disease (CBSD)
2. Cassava Green Mottle (CGM)
3. Cassava Mosaic Disease (CMD)
4. Healthy

Neste caso, o rótulo 3 - Cassava Mosaic Disease (CMD) (https://en.wikipedia.org/wiki/Cassava_mosaic_virus) é o rótulo mais comum. Esse desequilíbrio pode ter que ser tratado com uma função de perda ponderada ou sobreamostragem. E pode-se tentar isso em uma iteração futura deste kernel ou em um novo kernel.

Verificando uma imagem de exemplo para ver como ela se parece:

In [ ]:
from PIL import Image

img_cmd = Image.open(train_df['path'][1])
width, height = img_cmd.size
print(width,height) 

In [ ]:
img_cmd

## Carregando os dados

Depois de olhar os dados, os dados são carregados no fastai como objetos `DataLoaders`. 

Primeiro, vamos definir as transformações de item e em lote. As transformações de item realizam um corte bastante grande em cada uma das imagens, enquanto as transformações de lote realizam corte redimensionado aleatório para 512 e também aplicam outros aumentos padrão (em `aug_tranforms`) no nível de lote na GPU. O tamanho do lote é definido para 32 aqui.

In [ ]:
item_tfms = RandomResizedCrop(512, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=32

Embora o fastai forneça várias maneiras de fazer o carregamento de dados personalizado (até mesmo usando PyTorch DataLoaders simples), os problemas tradicionais de classificação de imagens funcionam bem na API de dados de alto nível. Aqui, são passadas todas as informações necessárias para criar um objeto `DataLoaders`

In [ ]:
dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               label_col=0, #label is in the first column of the DataFrame
                               fn_col=1, #filename/path is in the second column of the DataFrame
                               bs=bs, #pass in batch size
                               item_tfms=item_tfms, #pass in item_tfms
                               batch_tfms=batch_tfms) #pass in batch_tfms

Para confirmar a criação bem-sucedida do dataloader, podemos usar o comando `show_batch`, que mostra um subconjunto do lote:

In [ ]:
dls.show_batch()

## Treinando o modelo

Vamos treinar um modelo EfficientNet-B3 aa. Usando o pacote [timm](https://github.com/rwightman/pytorch-image-models) de Ross Wightman para definir o modelo. Como esta competição não permite acesso à Internet, adicionei os pesos pré-treinados de timm como um conjunto de dados, e a célula de código abaixo permitirá que timm encontre o arquivo:

In [ ]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/timmefficientnet/tf_efficientnet_b3-e3bd6955.pth' '/root/.cache/torch/hub/checkpoints/tf_efficientnet_b3-e3bd6955.pth'

No fastai, a classe de treinamento é o `Learner`, que recebe os dados, modelo, otimizador, função de perda, etc. e permite que você treine modelos, faça previsões, etc.

Ao treinar modelos CNN comuns como ResNets, normalmente podemos usar a função `cnn_learner` que cria um objeto `Learner` que nos permite treinar um modelo fornecido com os carregadores de dados fornecidos. No entanto, cnn_learner não oferece suporte aos modelos do timm prontos para uso. Zach Mueller (@muellerzr) [has written some simple functions](https://walkwithfastai.com/vision.external.timm)  escreveu algumas funções simples para tornar muito fácil criar objetos Learner para modelos timm.



In [ ]:
from timm import create_model
from fastai.vision.learner import _update_first_layer

def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")
        
def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

In [ ]:
def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

Vamos agora criar nosso objeto `Learner`. Também usando técnicas de treinamento de suavização de rótulos e otimizador `Ranger`, que são fornecidas no fastai. Também  usando a precisão mista com muita facilidade:

In [ ]:
learn = timm_learner(dls, 
                    'tf_efficientnet_b3_ns', 
                     opt_func = ranger,
                     loss_func=LabelSmoothingCrossEntropy(),
                     metrics = [accuracy]).to_native_fp16()


Agora temos um objeto `Learner` que tem um modelo congelado (apenas os pesos da cabeça do modelo podem ser atualizados). Para treinar um modelo, precisamos encontrar a taxa de aprendizagem ideal, o que pode ser feito com o localizador de taxa de aprendizagem do fastai `lr_find()`:

In [ ]:
learn.lr_find()

Frequentemente, se usa um modelo pré-treinado congelado para uma única época e, em seguida, treinar todo o modelo pré-treinado para várias épocas. O otimizador `Ranger` tem melhor desempenho com uma programação de taxa de aprendizado de recozimento plana + cosseno. Agora treinaremos o modelo congelado por uma época.

Conforme mostrado acima, a taxa de aprendizado ideal para treinar o modelo congelado é onde a perda está diminuindo mais rapidamente: cerca de ~ 1e-1. Por segurança, foi usado um peso elevado para ajudar a prevenir o sobreajuste. Também usaremos outra técnica comum de treinamento de última geração: `mixup`.

In [ ]:
learn.freeze() 
learn.fit_flat_cos(1,1e-1, wd=0.5, cbs=[MixUp()])

É fundamental que qualquer preparação de dados realizada em um conjunto de dados de treinamento também seja realizada em um novo conjunto de dados no futuro.
Isso pode incluir um conjunto de dados de teste ao avaliar um modelo ou novos dados do domínio ao usar um modelo para fazer previsões.
Normalmente, o modelo ajustado no conjunto de dados de treinamento é salvo para uso posterior. 

A solução correta para preparar novos dados para o modelo no futuro é também salvar quaisquer objetos de preparação de dados, como métodos de escalonamento de dados, para arquivar junto com o modelo.

Então salvamos o modelo usando o `save()` 

Isso armazena o modelo junto com os dados de treinamento usados para criá-lo.


In [ ]:
learn.save('modelo-1')

In [ ]:
learn = learn.load('modelo-1')

In [ ]:
learn.recorder.plot_loss()

Vamos agora descongelar o modelo e encontrar uma boa taxa de aprendizado `lr_find()`:

In [ ]:
learn.unfreeze()
learn.lr_find()

Vamos treinar por 10 épocas com o modelo descongelado.

In [ ]:
learn.unfreeze()
#learn.fit_one_cycle(10, max_lr=slice(0.0001737800776027143, 1.3182567499825382e-06))
learn.fit_flat_cos(10,2e-3,pct_start=0,cbs=[MixUp()])

Traçamos a perda `plot_loss`, colocamos o modelo de volta no [fp32](https://docs.fast.ai/callback.fp16.html).

A precisão anterior era de `fp16` (meia precisão)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn = learn.to_native_fp32()

Salavando e exportando o modelo para usar mais tarde, para possíveis inferências:

In [ ]:
learn.save('modelo-2')

In [ ]:
learn.export()

O `fastai`fornece métodos de interpretação para modelos de classificação, como o [ClassificationInterpretation](https://docs.fast.ai/interpret.html), para melhor interpretar as previsões de um modelo, e gerar a matriz de confusão:


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

## Inferência

A função `dls.test_dl` permite que você crie um dataloader de teste usando o mesmo pipeline definido anteriormente.

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df.head()

In [ ]:
_sample_df = sample_df.copy()
_sample_df['path'] = _sample_df['image_id'].map(lambda x:dataset_path/'test_images'/x)
_sample_df = _sample_df.drop(columns=['image_id'])
test_dl = dls.test_dl(_sample_df)

Visualizando o `test_dl`

In [ ]:
test_dl.show_batch()

Para as previsões, foi aplicada técnica Test-Time Augmentation (15x TTA), conforme [Jason Brownlee](https://machinelearningmastery.com/how-to-use-test-time-augmentation-to-improve-model-performance-for-image-classification/#:~:text=Test%2Dtime%20augmentation%2C%20or%20TTA,an%20ensemble%20of%20those%20predictions).

In [ ]:
preds, _ = learn.tta(dl=test_dl, n=10, beta=0)

Preparando submissão

In [ ]:
sample_df['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample_df.to_csv('submission.csv',index=False)